In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index


array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '8' encode 6
defense '10' encode 7


In [5]:
x_test_adv_0_1.shape

(1432518, 42)

In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb100 = XGBClassifier()
xgb100.fit(x_test_adv_0_1, y_label_0_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf100 = RandomForestClassifier()
rf100.fit(x_test_adv_0_1, y_label_0_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt100 = DecisionTreeClassifier()
# dt100.fit(x_test_adv_0_1, y_label_0_1)

xgb
RF


RandomForestClassifier()

In [17]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [18]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb100.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input100/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input100/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 4 5 6] [     9      2      2      1 238739]
(9, 42) (9,)
Save 2 to baseline_Def2.npy
(2, 42) (2,)
Save 3 to baseline_Def3.npy
(2, 42) (2,)
Save 4 to baseline_Def4.npy
(1, 42) (1,)
Save 5 to baseline_Def5.npy
(238739, 42) (238739,)
Save 6 to baseline_Def6.npy
Execution Time: 1.135676 seconds
(716259,)
[1 2 3 4 5 6] [284628      2 213829     55     39 217706]
(284628, 42) (284628,)
Save 1 to BIM_Def1.npy
(2, 42) (2,)
Save 2 to BIM_Def2.npy
(213829, 42) (213829,)
Save 3 to BIM_Def3.npy
(55, 42) (55,)
Save 4 to BIM_Def4.npy
(39, 42) (39,)
Save 5 to BIM_Def5.npy
(217706, 42) (217706,)
Save 6 to BIM_Def6.npy
Execution Time: 3.160395 seconds
(716259,)
[1 2 3 4 5 6] [494789     44   2854    254     88 218230]
(494789, 42) (494789,)
Save 1 to FGSM_Def1.npy
(44, 42) (44,)
Save 2 to FGSM_Def2.npy
(2854, 42) (2854,)
Save 3 to FGSM_Def3.npy
(254, 42) (254,)
Save 4 to FGSM_Def4.npy
(88, 42) (88,)
Save 5 to FGSM_Def5.npy
(218230, 42) (218230,)
Save 6 to FGSM_Def6.npy
Execution Time: 3.

In [19]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf100.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input100/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input100/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 4 5 6] [    10      1      1      1 238740]
(10, 42) (10,)
Save 2 to baseline_Def2.npy
(1, 42) (1,)
Save 3 to baseline_Def3.npy
(1, 42) (1,)
Save 4 to baseline_Def4.npy
(1, 42) (1,)
Save 5 to baseline_Def5.npy
(238740, 42) (238740,)
Save 6 to baseline_Def6.npy
Execution Time: 3.853939 seconds
(716259,)
[1 2 3 4 5 6] [451746 233721  26497      2      4   4289]
(451746, 42) (451746,)
Save 1 to BIM_Def1.npy
(233721, 42) (233721,)
Save 2 to BIM_Def2.npy
(26497, 42) (26497,)
Save 3 to BIM_Def3.npy
(2, 42) (2,)
Save 4 to BIM_Def4.npy
(4, 42) (4,)
Save 5 to BIM_Def5.npy
(4289, 42) (4289,)
Save 6 to BIM_Def6.npy
Execution Time: 12.868100 seconds
(716259,)
[1 2 3 4 6] [478301 231031     23      5   6899]
(478301, 42) (478301,)
Save 1 to FGSM_Def1.npy
(231031, 42) (231031,)
Save 2 to FGSM_Def2.npy
(23, 42) (23,)
Save 3 to FGSM_Def3.npy
(5, 42) (5,)
Save 4 to FGSM_Def4.npy
(6899, 42) (6899,)
Save 6 to FGSM_Def6.npy
Execution Time: 8.431970 seconds
(716259,)
[1 2 3 4 5 6] [451746 23

In [20]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt100.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input100/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input100/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [21]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}100")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Exclude3Attack/WUSTL_Input100/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [22]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input100_Exclude3Attack.csv")

In [23]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB100,0,9,2,2,1,238739,0,0,0,0,0,238753
1,XGB100,284628,2,213829,55,39,217706,0,0,0,0,0,716259
2,XGB100,494789,44,2854,254,88,218230,0,0,0,0,0,716259
3,XGB100,284628,2,213829,55,39,217706,0,0,0,0,0,716259
4,XGB100,709992,9,5491,8,700,59,0,0,0,0,0,716259
5,XGB100,381890,7,100129,4,331,233897,0,1,0,0,0,716259
6,XGB100,0,34,5,7,7,716206,0,0,0,0,0,716259
7,XGB100,227555,94107,13508,40,190,380859,0,0,0,0,0,716259
8,XGB100,23179,0,473195,79,47,219759,0,0,0,0,0,716259
9,XGB100,495447,0,6755,194,78,213784,0,0,0,1,0,716259
